# **Il progetto**

Il presente progetto si propone di analizzare i documenti informativi precontrattuali (DIP) relativi alle polizze di responsabilità civile auto (RCA), con particolare attenzione alla valutazione della loro chiarezza espositiva. L’obiettivo principale è fornire uno strumento in grado di effettuare un’analisi preliminare (ex ante) della comprensibilità dei documenti da parte di utenti non esperti.

Considerato l’obbligo normativo per le compagnie assicurative di presentare il DIP, lo strumento proposto può rappresentare un supporto utile per IVASS nell'attività di valutazione della qualità dei documenti prodotti. Infatti, pur esistendo linee guida sulla redazione del DIP, non sempre tali documenti risultano essere redatti in modo chiaro e accessibile.

A tal fine, è stato sviluppato un agente dotato di un tool multi-input per l’estrazione mirata di porzioni di testo (chunk) rilevanti dallo spazio vettoriale. Per ottimizzare l’accuratezza dell’analisi, l’agente è stato potenziato con esempi esplicativi e contestualizzato mediante un system prompt che ne definisce il ruolo e il contesto operativo.

# **1**

**Inizializzazione dell'LLM, upload e vettorizzazione dei documenti**

In questa cella installiamo la libreria langchain per Google Gemini e pypdf ci permette di avere un llm in grado di leggere i file pdf.

In [ ]:
%pip install -q langchain-google-genai langchain_community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 891.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


Qui importiamo le librerie che useremo nelle prossime celle.

In [75]:
import os
import bs4
from google.colab import files
from langchain import hub
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools import tool
from langchain.agents import initialize_agent, AgentType

import time
import pandas as pd

Inserimento della chiave per utilizzare Gemini

In [ ]:
os.environ['GOOGLE_API_KEY'] ="******************+"

Definizione del modello llm che utilizzeremo, in particolare, abbiamo scelto di utilizzare il modello con temperatura = 0, ciò per avere risposte consistenti

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                             temperature=0.0,
                             #max_tokens = 200,
                             )

Qui definiamo lo spazio di embedding ed inizializziamo un database vettorizzato per lo store dei DIP

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
vector_store_pdfs = InMemoryVectorStore(embeddings)

Inizializziamo la classe text_splitter, che utilizzeremo per dividere i pdf in chunk di dimensione 1000 con una sovrapposizione, quello prima con quello dopo, pari a 200.

In [ ]:
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
  )

Qui si caricano i DIP

In [ ]:
uploaded = files.upload()

Ora, per ottimizzare il tool che utilizzeremo, ossia un RAG tool, carichiamo i nostri file, dopo averli divisi in chunk nello spazio vettorizzato riportando in ogni chunk anche il metadata del nome. Questo per evitare che il nostro agente prenda chunk da file diversi.

In [60]:
for filename in uploaded.keys():
  loader = PyPDFLoader(filename)
  docs = loader.load()

  for doc in docs:
    doc.metadata["source"] = filename #per tenere traccia del pdf di provenienza

  all_splits = text_splitter.split_documents(docs)
  vector_store_pdfs.add_documents(all_splits)

Saving allianz_rca.pdf to allianz_rca.pdf
Saving ande_rca.pdf to ande_rca.pdf
Saving arca_rca.pdf to arca_rca.pdf
Saving assimoco_rca.pdf to assimoco_rca.pdf
Saving axa_rca.pdf to axa_rca.pdf
Saving bene_rca.pdf to bene_rca.pdf
Saving generali_rca.pdf to generali_rca.pdf
Saving groupama_rca.pdf to groupama_rca.pdf
Saving prima_rca.pdf to prima_rca.pdf
Saving sara_rca.pdf to sara_rca.pdf
Saving unipol_rca.pdf to unipol_rca.pdf


# **2**

**RAG tool, agente e chain**

Qui definiamo il tool che il nostro agente utilizzerà. Questo ha due input:

1. La query, che farà allo spazio vettorizzato per recuperare le informazioni attinenti al prompt.

2. Il document_name, il quale è facoltativo e serve ad filtrare i chunk ritornati dalla query, in modo che abbia solo quelli relativi al documento in questione.

In [88]:
@tool
def rag_tool(query: str, document_name: str = None):
    """
    Ritorna le informazioni semanticamente rilevanti per la query.
    Si può dare l'opzione di cercare su un documento specifico
    """
    retrieved_docs = vector_store_pdfs.similarity_search(query, k=20)

    # tra tutti i chunk ritornati, cerca quelli che hanno nei metadati il document_name
    if document_name:
        retrieved_docs = [doc for doc in retrieved_docs if doc.metadata.get("source") == document_name]
    # caso nel cui non sia trovato il document name
    if not retrieved_docs:
        return "No matching documents found."

    #aggregazione dei chunk
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return docs_content

Contesto nel quale il nostro agente si dovrà calare.

In [77]:
system_context = """
                       Sei un neofita in tematiche assicurative.
                       Devi valutare la chiarezza del DIP (documento informativo precontrattuale) di contratti RCA
                       (responsabilità civile auto).
                       Devi tenere in considerazione i seguenti aspetti:
                       1) I termini assicurativi specifici utilizzati sono stati definiti, ossia spiegati in precedenza.
                       2) La forma logica e grammaticale delle frasi è corretta e semplice.
                       Ricorda che il tuo obiettivo è valutare la chiarezza del documento e puoi rispondere solo ed esclusivamente
                       con un valore tra 0 e 10.
                 """

Tre esempi di valutazioni di DIP, la risposta (answer) è il pensiero che l'agente formula dopo aver recuperato le informazioni dai chunk pertinenti.

In [78]:
examples = [

    {
        "nome_ass":"assicurazione1_rca.pdf",

        "answer": """
                  Che cosa è assicurato?
                  - Descrizione chiara e completa delle coperture: danni a terzi, trasportati, circolazione in aree pubbliche e private, massimali minimi e possibilità di massimali superiori.
                  - Garanzie opzionali elencate con chiarezza e distinzione dal contratto base.
                  Che cosa non è assicurato?
                  - Elenco dettagliato e ben organizzato delle esclusioni con simboli grafici e spiegazioni semplici.
                  - Chiarezza su chi non è considerato terzo e sui danni esclusi.
                  Ci sono limiti di copertura?
                  - Spiegazione esaustiva delle rivalse con esempi concreti (guida senza patente, stato di ebbrezza, gare sportive).
                  - Indicazione chiara delle conseguenze per l’assicurato, con avvertenze evidenziate.""",

                  "evaluation": '8'
    },

    {
        "nome_ass":"assicurazione2_rca.pdf",
        "answer":"""
                  Che cosa è assicurato?
                  - Descrizione vaga, ad esempio: "Copre i danni causati dalla circolazione del veicolo."
                  - Nessuna indicazione chiara sui massimali o sulle specifiche coperture (es. danni a persone, cose, trasportati).
                  Che cosa non è assicurato?
                  - Elenco confuso o assente delle esclusioni.
                  - Mancanza di chiarezza su chi non è considerato terzo o su quali danni non sono coperti.
                  Ci sono limiti di copertura?
                  - Nessuna o scarsa spiegazione delle rivalse o delle condizioni che limitano la copertura.
                  - Linguaggio tecnico non spiegato o assenza di esempi pratici.""",

        "evaluation":"3"
    },

    {
        "nome_ass":"assicurazione3_rca.pdf",
        "answer":""" Che cosa è assicurato?
                    - Elenco delle coperture base (danni a terzi, massimali minimi di legge) ma con linguaggio poco fluido
                     e qualche termine tecnico non spiegato.
                    - Alcune garanzie opzionali menzionate senza dettagli.
                    Che cosa non è assicurato?
                    - Elenco delle esclusioni presente ma non ben organizzato, con frasi lunghe e poco chiare.
                    - Mancanza di esempi o di evidenziazioni grafiche per facilitare la lettura.
                    Ci sono limiti di copertura?
                    - Indicazione delle rivalse presente ma con spiegazioni sintetiche e poco esplicative.
                    - Mancanza di esempi concreti o di avvertenze evidenti.""",
      "evaluation":"6"
    }
]

Inizializzazione dell'agente, del prompt e della chain

In [106]:
tools = [rag_tool]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
)

prompt = ChatPromptTemplate.from_messages(
      [
          ("system",system_context),
          ("human", f"Valuta la chiarezza del dip di {examples[0]['nome_ass']}. Rispondi solo con la tua valutazione da 0 a 10"),
          ("ai", f"###Thought: {examples[0]['answer']} ### Output: {examples[0]['evaluation']}"),
          ("human", f"Valuta la chiarezza del dip di {examples[1]['nome_ass']}. Rispondi solo con la tua valutazione da 0 a 10"),
          ("ai", f"###Thought: {examples[1]['answer']} ### Output: {examples[1]['evaluation']}"),
          ("human", f"Valuta la chiarezza del dip di {examples[2]['nome_ass']}. Rispondi solo con la tua valutazione da 0 a 10"),
          ("ai", f"###Thought: {examples[2]['answer']} ### Output: {examples[2]['evaluation']}"),
          ("human", "Qual'è la tua valutazione sulla chiarezza di questo documento informativo precontrattuale? {input}")
      ]
      )


chain = prompt | agent

Loop sui nomi dei file per analizzarli uno alla volta. Il nome è passato come input all'agente. Le risposte sono salvate in un pandas DataFrame. Se non si vuole vedere le osservazioni e il pensiero dell'agente dip per dip, si deve impostare verbose = False  nell'inizializzazione dell'agente nella precente cella.

In [116]:
response = pd.DataFrame({
    'DIP': list(uploaded.keys()),
    'Valutazione': [0] * len(uploaded)
})

for ass in uploaded.keys():

  print(f'DIP {ass} in fase di valutazione.\n')

  output = 'a'
  counter = 3

  while not (output.isdigit()):

    output = chain.invoke({'input': ass})['output']
    counter = counter -1

    if counter < 1:
      break

  response.loc[response['DIP'] == ass, 'Valutazione'] = output


  time.sleep(10)

DIP allianz_rca.pdf in fase di valutazione.



<ipython-input-116-3027189941>:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  response.loc[response['DIP'] == ass, 'Valutazione'] = output


DIP ande_rca.pdf in fase di valutazione.

DIP arca_rca.pdf in fase di valutazione.

DIP assimoco_rca.pdf in fase di valutazione.

DIP axa_rca.pdf in fase di valutazione.

DIP bene_rca.pdf in fase di valutazione.

DIP generali_rca.pdf in fase di valutazione.

DIP groupama_rca.pdf in fase di valutazione.

DIP prima_rca.pdf in fase di valutazione.

DIP sara_rca.pdf in fase di valutazione.

DIP unipol_rca.pdf in fase di valutazione.



,DIP,Valutazione
0,allianz_rca.pdf,5
1,ande_rca.pdf,7
2,arca_rca.pdf,5
3,assimoco_rca.pdf,7
4,axa_rca.pdf,5
5,bene_rca.pdf,7
6,generali_rca.pdf,7
7,groupama_rca.pdf,6
8,prima_rca.pdf,7
9,sara_rca.pdf,6


In [118]:
response

,DIP,Valutazione
0,allianz_rca.pdf,5
1,ande_rca.pdf,7
2,arca_rca.pdf,5
3,assimoco_rca.pdf,7
4,axa_rca.pdf,5
5,bene_rca.pdf,7
6,generali_rca.pdf,7
7,groupama_rca.pdf,6
8,prima_rca.pdf,7
9,sara_rca.pdf,6
